In [28]:
import pm4py
import pandas as pd

In [29]:
log = pm4py.read_xes("../data/BPI_Challenge_2017.xes.gz")
dataframe_initial = pm4py.convert_to_dataframe(log)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [30]:
df = dataframe_initial.copy()

In [31]:
df.columns

Index(['Action', 'org:resource', 'concept:name', 'EventOrigin', 'EventID',
       'lifecycle:transition', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID'],
      dtype='object')

In [32]:
df["time:timestamp"]

0         2016-01-01 09:51:15.304000+00:00
1         2016-01-01 09:51:15.352000+00:00
2         2016-01-01 09:51:15.774000+00:00
3         2016-01-01 09:52:36.392000+00:00
4         2016-01-01 09:52:36.403000+00:00
                        ...               
1202262   2017-01-06 06:33:02.212000+00:00
1202263   2017-01-06 06:33:02.221000+00:00
1202264   2017-01-16 09:51:21.114000+00:00
1202265   2017-01-16 09:51:21.139000+00:00
1202266   2017-01-16 09:51:21.146000+00:00
Name: time:timestamp, Length: 1202267, dtype: datetime64[ns, UTC]

In [33]:
df["concept:name"]

0            A_Create Application
1                     A_Submitted
2                  W_Handle leads
3                  W_Handle leads
4          W_Complete application
                    ...          
1202262       W_Call after offers
1202263       W_Call after offers
1202264               A_Cancelled
1202265               O_Cancelled
1202266       W_Call after offers
Name: concept:name, Length: 1202267, dtype: object

In [47]:
# event_of_interest = "A_Cancelled"
event_of_interest = "O_Accepted"

In [48]:
# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['time:timestamp'])
df['case_id'] = df["case:concept:name"].apply(lambda x: int(x.split("_")[1]))
df["event_name"] = df["concept:name"]

# Sort by caseID and timestamp
df = df.sort_values(['case_id', 'timestamp'])

# Get the first timestamp for each caseID
start_times = df.groupby('case_id')['timestamp'].min().rename('start_time')

# Get the "A_Cancelled" timestamp for each caseID
cancelled_times = df[df['event_name'] == event_of_interest].groupby('case_id')['timestamp'].min().rename('cancelled_time')

# Combine start and cancelled times
time_df = pd.concat([start_times, cancelled_times], axis=1).dropna()

# Calculate time difference
time_df['time_until_cancelled'] = time_df['cancelled_time'] - time_df['start_time']

# Convert timedelta to total seconds to avoid overflow
time_df['time_until_cancelled_seconds'] = time_df['time_until_cancelled'].dt.total_seconds()

# Total time until "A_Cancelled" across all cases in seconds
total_time_seconds = time_df['time_until_cancelled_seconds'].sum()
average_time_per_case = total_time_seconds/len(time_df)

# Optionally, convert total seconds to a more readable format (e.g., hours, days)
total_time_hours = average_time_per_case / 3600  # Convert to hours
total_time_days = average_time_per_case / (3600 * 24)  # Convert to days

print(f"Total time until '{event_of_interest}' across all cases: {total_time_seconds} seconds")
print(f"Total time until '{event_of_interest}' per case: {average_time_per_case} seconds")
print(f"Which is approximately {total_time_hours:.2f} hours or {total_time_days:.2f} days.")

Total time until 'O_Accepted' across all cases: 26959345901.808 seconds
Total time until 'O_Accepted' per case: 1564856.3908641746 seconds
Which is approximately 434.68 hours or 18.11 days.


Total time until 'A_Cancelled' across all cases: 26908165328.758 seconds <br>
Total time until 'A_Cancelled' per case: 2579634.294771163 seconds <br>
Which is approximately 716.57 hours or 29.86 days. <br>
 <br>
Total time until 'O_Accepted' across all cases: 26959345901.808 seconds <br>
Total time until 'O_Accepted' per case: 1564856.3908641746 seconds <br>
Which is approximately 434.68 hours or 18.11 days. <br>

In [64]:
import pandas as pd

df = dataframe_initial.copy()

# Assuming you have already loaded your DataFrame 'df' with the necessary columns

# Define the event of interest
event_of_interest = "O_Accepted"

# Step 1: Data Preparation
# ------------------------

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['time:timestamp'])

# Extract case_id from "case:concept:name"
df['case_id'] = df["case:concept:name"].apply(lambda x: int(x.split("_")[1]))

# Rename "concept:name" to "event_name" for clarity
df["event_name"] = df["concept:name"]

# Step 2: Sort Data
# -----------------

# Sort by case_id and timestamp to ensure chronological order within each case
df = df.sort_values(['case_id', 'timestamp'])

# Step 3: Define a Function to Count Events Before "A_Cancelled"
# -------------------------------------------------------------

def count_events_before_cancelled(group, event_name):
    """
    Counts the number of events that occur before the first occurrence of 'event_name' within a group.
    
    Parameters:
    - group (DataFrame): The subset of the DataFrame corresponding to a single case_id.
    - event_name (str): The name of the event to search for (e.g., "A_Cancelled").
    
    Returns:
    - int or pd.NA: The count of events before 'event_name' or pd.NA if 'event_name' does not occur.
    """
    # Reset index to ensure proper ordering
    group = group.reset_index(drop=True)
    
    # Find the index of the first occurrence of 'event_name'
    cancelled_indices = group[group['event_name'] == event_name].index
    
    if not cancelled_indices.empty:
        first_cancelled_idx = cancelled_indices[0]
        # Number of events before 'A_Cancelled' is the index of 'A_Cancelled'
        return first_cancelled_idx
    else:
        # If 'A_Cancelled' does not exist, return pd.NA or another indicator
        return pd.NA

# Step 4: Apply the Function to Each Group
# ----------------------------------------

# Group the DataFrame by 'case_id' and apply the counting function
event_counts = df.groupby('case_id').apply(lambda group: count_events_before_cancelled(group, event_of_interest)).rename('events_before_A_Cancelled')

# Step 5: Handle Cases Without "A_Cancelled"
# ------------------------------------------

# Optionally, drop cases where 'A_Cancelled' does not occur
event_counts = event_counts.dropna()

# Convert counts to integer type (since pd.NA introduces float)
event_counts = event_counts.astype(int)
print(event_counts)

# Step 6: Aggregate the Counts
# ----------------------------

# Total number of events before 'A_Cancelled' across all cases
total_events_before_cancelled = event_counts.sum()

# Average number of events before 'A_Cancelled' per case
average_events_per_case = total_events_before_cancelled / len(event_counts)

print(len(event_counts))
# Step 7: Output the Results
# --------------------------

print(f"Number of events before '{event_of_interest}' for each case_id:")
print(event_counts)

print(f"\nTotal number of events before '{event_of_interest}' across all cases:", total_events_before_cancelled)
print(f"Average number of events before '{event_of_interest}' per case: {average_events_per_case:.2f}")

case_id
235300         63
355337         29
438333        107
919303         20
1030996        35
             ... 
2146802304     52
2147069097     34
2147147129     47
2147201499     32
2147356192     62
Name: events_before_A_Cancelled, Length: 17228, dtype: int64
17228
Number of events before 'O_Accepted' for each case_id:
case_id
235300         63
355337         29
438333        107
919303         20
1030996        35
             ... 
2146802304     52
2147069097     34
2147147129     47
2147201499     32
2147356192     62
Name: events_before_A_Cancelled, Length: 17228, dtype: int64

Total number of events before 'O_Accepted' across all cases: 721670
Average number of events before 'O_Accepted' per case: 41.89


C:\Users\20202034\AppData\Local\Temp\ipykernel_26868\697826585.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  event_counts = df.groupby('case_id').apply(lambda group: count_events_before_cancelled(group, event_of_interest)).rename('events_before_A_Cancelled')


Total number of events before 'A_Cancelled' across all cases: 239868 <br>
Average number of events before 'A_Cancelled' per case: 23.00 <br>
 <br>
Total number of events before 'O_Accepted' across all cases: 721670 <br>
Average number of events before 'O_Accepted' per case: 41.89 <br>

In [56]:
dataframe_initial["case:concept:name"]

0           Application_652823628
1           Application_652823628
2           Application_652823628
3           Application_652823628
4           Application_652823628
                    ...          
1202262    Application_1350494635
1202263    Application_1350494635
1202264    Application_1350494635
1202265    Application_1350494635
1202266    Application_1350494635
Name: case:concept:name, Length: 1202267, dtype: object

In [ ]:
import pandas as pd

df = dataframe_initial.copy()

# Assuming you have already loaded your DataFrame 'df' with the necessary columns

# Define the event of interest
event_of_interest = "A_Cancelled"

# Step 1: Data Preparation
# ------------------------

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['time:timestamp'])

# Extract case_id from "case:concept:name"
df['case_id'] = df["case:concept:name"].apply(lambda x: int(x.split("_")[1]))

# Rename "concept:name" to "event_name" for clarity
df["event_name"] = df["concept:name"]

case_ids_event = df[df["event_name"] == event_of_interest].case_id.tolist()
df_event = df[df["case_id"].isin(case_ids_event)]
case_counts_event = df_event["case_id"].value_counts()
total_events_before_cancelled = case_counts_event.sum()

# Average number of events before 'A_Cancelled' per case
average_events_per_case = total_events_before_cancelled / len(case_counts_event)

print(f"Number of events before '{event_of_interest}' for each case_id:")
print(case_counts_event)

print(f"\nTotal number of events before '{event_of_interest}' across all cases:", total_events_before_cancelled)
print(f"Average number of events before '{event_of_interest}' per case: {average_events_per_case:.2f}")

# NOTE: NOT CORRECT, this counts complete trace length not until the event of interest

Number of events before 'A_Cancelled' for each case_id:
case_id
1031629108    150
450313645     146
2018258104    142
773585477     127
1631137033    124
             ... 
228161231      10
1781717045     10
961957338      10
6992306        10
1829767138     10
Name: count, Length: 10431, dtype: int64

Total number of events before 'A_Cancelled' across all cases: 273749
Average number of events before 'A_Cancelled' per case: 26.24


In [40]:
df = dataframe_initial.copy()

# Calculate frequencies
case_id_counts = df["case:concept:name"].value_counts()

# Find the 10 least frequent case_ids
least_frequent = case_id_counts.nsmallest(10)

# Convert to DataFrame
least_frequent_df = least_frequent.reset_index()
least_frequent_df.columns = ["case:concept:name", "count"]

print("10 Least Frequent case_ids:")
print(least_frequent_df)

10 Least Frequent case_ids:
        case:concept:name  count
0   Application_419883832     10
1   Application_523628252     10
2   Application_104674625     10
3  Application_1823648369     10
4   Application_720418044     10
5  Application_1829767138     10
6   Application_228161231     10
7  Application_1781717045     10
8   Application_961957338     10
9   Application_519478762     10


Idea:

- since A_Cancelled takes longer 50% longer, but has on average about half of the events, the average time between events for Cancelled should be way longer, this could then be a good predictor
- Only problem, pretty likely that all this extra time comes form only waiting for cancelled. So if this is the only extra time this does not help us a lot in the earlier events. 